In [ ]:
!pip3 install psaw
!pip3 install praw
!pip3 install sqlalchemy
!pip3 install pandas
!pip3 install import-ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
import re
import string
import praw
import sys
import html
import pandas as pd
import logging as lg
from datetime import datetime as dt, timedelta
from psaw import *
from random import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.exc import InvalidRequestError, IntegrityError
from sqlalchemy import Column, Integer, String, create_engine, ForeignKey, UniqueConstraint, Index
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func, desc

In [ ]:
lg.basicConfig(encoding='utf-8', level=logging.INFO)

In [ ]:
punc = '''!()-[]|{};:\<>/@#$%^&*_~'''

relp = " "*len(punc)
def process(s):
  result = s.encode("ascii", "ignore").decode()
  result = html.unescape(result)
  result = re.sub(r"http\S+", '', result, flags=re.MULTILINE)
  result = result.translate(str.maketrans(punc,relp))
  result = re.sub(' +', ' ',result)
  return result;

In [ ]:
lg.debug(process("àa&string__withé|fuünny@charactersß. 32232*(&(*&^&*)) [ eww https://ww.fooodlc.di efew]"))

In [ ]:
#Models
Base = declarative_base()
class Submission(Base):
      __tablename__ = 'submissions'
      id = Column(Integer, primary_key=True)
      reddit_id = Column(String,nullable=False,index=True)
      subreddit = Column(String)
      title = Column(String)
      body = Column(String)
      link = Column(String)
      score = Column(Integer)
      created_utc = Column(Integer)
      __table_args__ = (UniqueConstraint('reddit_id'),)

      def __repr__(self):
          return f"{self.title[:100]} {self.body[:100]}\nscore:{self.score}"
class Comment(Base):
      __tablename__ = 'comments'
      id = Column(Integer, primary_key=True)
      reddit_id = Column(String,nullable=False,index=True)
      submission_id = Column(Integer, ForeignKey('submissions.id'),nullable=False,index=True)
      subreddit = Column(String)
      body = Column(String)
      score = Column(Integer)
      __table_args__ = (UniqueConstraint('reddit_id'),)

      def __repr__(self):
          return f'{self.body}'


def get_session(db_name):
  engine = create_engine(f'sqlite:////content/drive/MyDrive/datasets/reddit/{db_name}.db', echo=False)
  Session = sessionmaker(bind=engine)
  session = Session()
  Base.metadata.create_all(engine)
  return session

In [ ]:
reddit = praw.reddit.Reddit(client_id="", client_secret="",password="",user_agent="",username="")
api = PushshiftAPI(reddit)
session = get_session("datasets")

In [ ]:
#scraping only posts from pushshift api. Comments from bigquery
for sub in subs_arr:
  fetch_and_add_submissions(sub = sub,num_posts=100,interval = 10)
  lg.info(f"Done: {sub}")
                          

In [ ]:
#Subs to scrape data from.
subs_arr = ["politics","PoliticalHumor","PoliticalCompassMemes","Conservative",
            "TheRightCantMeme","neoliberal","democrats","PoliticalDiscussion",
            "republicans","AskReddit","Showerthoughts",
            "unpopularopinion","explainlikeimfive","askscience","Economics",
            "Libertarian""AskEconomics","badeconomics","ProgrammerHumor",
            "interestingasfuck","AskWomen","OldSchoolCool","aww","funny"]


#Scrape posts using pushshift api.
def fetch_and_add_submissions(sub,num_posts,
                              sort="desc",sort_type="num_comments",
                              save_as = "datasets",interval = 10, start_ts = dt.now().timestamp(),
                              end_ts =  dt(2011, 1, 1).timestamp(),
                              before = 0)):
  end = int(start_ts-end_ts)/(24*60*60)
  while(before<end):
    error_count = 0
    after = before + interval
    lg.info(f"after: {after},before: {before}")
    res = list(api.search_submissions(subreddit=sub,limit=500,after=f"{after}d",before=f"{before}d",sort_type=sort_type,sort=sort))
    res = list(filter(lambda x:x.score>10,res))
    time = dt.now().timestamp()

    for i in sorted(res,key=lambda x:x.score,reverse=True)[:num_posts]:
      subreddit = i.subreddit.display_name
      link = "" if f"https://www.reddit.com/r/{subreddit}" in i.url else i.url
      
      post = Submission(title=process(i.title), body=process(i.selftext),reddit_id=i.id,
                        score=i.score,link=link,subreddit=subreddit,created_utc=int(i.created_utc))
      
      if session.query(Submission.reddit_id).filter(Submission.reddit_id==post.reddit_id).count() == 0:
        session.add(post)
      else:
        error_count += 1
    try:
        session.commit()
    except:
        lg.error("Error Commiting")

    lg.info(f"Errors: {error_count}/{min(num_posts,len(res))}")
    before += interval

  lg.info(f"Done: {subs}")

In [ ]:
#scrape posts using ids
def fetch_and_add_submissions_by_id(ids):
  to_add = []
  for i in ids:
    if session.query(Submission).filter(Submission.reddit_id==i).count()==0:
      to_add.append(i)
  if(len(to_add)==0):
    return
  error_count = 0
  
  res = list(api.search_submissions(ids=",".join(to_add)))

  for i in res:
    subreddit = i.subreddit.display_name
    link = "" if f"https://www.reddit.com/r/{subreddit}" in i.url else i.url
    post = Submission(title=process(i.title), body=process(i.selftext),reddit_id=i.id,
                        score=i.score,link=link,subreddit=subreddit,created_utc=int(i.created_utc))
      
    if session.query(Submission.reddit_id).filter(Submission.reddit_id==post.reddit_id).count() == 0:
      session.add(post)
    else:
      error_count += 1

  try:
      session.commit()
  except:
      error_count+=1
  lg.info(f"Errors: {error_count}/{len(res)}")



In [ ]:
#scrape comments using pushshift
def fetch_and_add_comments(subs,
                              sort="desc",sort_type="score",
                              save_as = "datasets",interval = 2, start_ts = dt.now().timestamp(),
                              end_ts =  dt(2011, 1, 1).timestamp(),
                              before = 0):
  end = int(start_ts-end_ts)/(60*60)
  while(before<end):
    error_count = 0
    after = before + interval
    lg.info(f"after: {after},before: {before}")
    
    res = list(api.search_comments(subreddit=subs,after=f"{after}h",limit=500,before=f"{before}h",sort_type=sort_type,sort=sort))
    res = list(filter(lambda x: x.score>10,res))    

    for i in res:
      subreddit = i.subreddit.display_name
      comment = Comment(body=process(i.body),reddit_id=i.id,
                        score=i.score,subreddit=subreddit,submission_id=i.link_id[3:])
      
      if session.query(Submission).filter(Submission.reddit_id==comment.submission_id).count()!=0 and session.query(Comment.reddit_id).filter(Comment.reddit_id==comment.reddit_id).count() == 0:
        session.add(comment)
      else:
        error_count += 1
      
    try:
        session.commit()
    except:
        lg.error("Error Commiting")

    lg.info(f"Errors: {error_count}/{len(res)}")
    before += interval

  lg.info(f"Done {subs}")